# Natural Language Processing

## Datasets, Preprocessing, Streaming

## 1. Working on your own file

We’ll show you how Datasets can be used to load datasets that aren’t available on the Hugging Face Hub.

### Working with local and remote datasets

Datasets provides loading scripts to handle the loading of local and remote datasets.

|Data format	   | Loading script	    |Example                                                 |
| ---              | ---                | ---                                                    |
|CSV & TSV         | csv                | `load_dataset("csv", data_files="my_file.csv")`        |
|Text files	       | text	            | `load_dataset("text", data_files="my_file.txt")`       |
|JSON & JSON Lines | json	            | `load_dataset("json", data_files="my_file.jsonl")`     |
|Pickled DataFrames|pandas	            | `load_dataset("pandas", data_files="my_dataframe.pkl")`|

### Loading a local dataset

For this example we’ll use some drug effectiveness dataset:

You can use the `wget` in your terminal:

    wget -P data/ "https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip"

In [1]:
#comment this if you are using puffer or tokyo
import os
os.environ['http_proxy'] = 'http://192.41.170.23:3128'
os.environ['https_proxy'] = 'http://192.41.170.23:3128'

Since TSV is just a variant of CSV that uses tabs instead of commas as the separator, we can load these files by using the csv loading script and specifying the delimiter argument in the load_dataset() function as follows:

In [2]:
from datasets import load_dataset

drug_dataset = load_dataset("csv", data_files="data/drugsComTrain_raw.tsv", delimiter="\t")

Using custom data configuration default-9f769bd66dd9ad28
Found cached dataset csv (/home/chaklams/.cache/huggingface/datasets/csv/default-9f769bd66dd9ad28/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

By default, loading local files creates a `DatasetDict` object with a train split. We can see this by inspecting the `drug_dataset` object:

In [3]:
drug_dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 161297
    })
})

This shows us the number of rows and the column names associated with the training set.

A good practice when doing any sort of data analysis is to grab a small random sample to get a quick feel for the type of data you’re working with. In 🤗 Datasets, we can create a random sample by chaining the `Dataset.shuffle()` and `Dataset.select()` functions together:

In [4]:
drug_sample = drug_dataset["train"].shuffle(seed=42).select(range(1000))
# Peek at the first few examples
drug_sample[:3]

Loading cached shuffled indices for dataset at /home/chaklams/.cache/huggingface/datasets/csv/default-9f769bd66dd9ad28/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-370401d6b4fc92a0.arrow


{'Unnamed: 0': [87571, 178045, 80482],
 'drugName': ['Naproxen', 'Duloxetine', 'Mobic'],
 'condition': ['Gout, Acute', 'ibromyalgia', 'Inflammatory Conditions'],
 'review': ['"like the previous person mention, I&#039;m a strong believer of aleve, it works faster for my gout than the prescription meds I take. No more going to the doctor for refills.....Aleve works!"',
  '"I have taken Cymbalta for about a year and a half for fibromyalgia pain. It is great\r\nas a pain reducer and an anti-depressant, however, the side effects outweighed \r\nany benefit I got from it. I had trouble with restlessness, being tired constantly,\r\ndizziness, dry mouth, numbness and tingling in my feet, and horrible sweating. I am\r\nbeing weaned off of it now. Went from 60 mg to 30mg and now to 15 mg. I will be\r\noff completely in about a week. The fibro pain is coming back, but I would rather deal with it than the side effects."',
  '"I have been taking Mobic for over a year with no side effects other than 

Great, we’ve loaded our first local dataset! But while this worked for the training set, what we really want is to include both the train and test splits in a single DatasetDict object so we can apply `Dataset.map()` functions across both splits at once. To do this, we can provide a dictionary to the `data_files` argument that maps each split name to a file associated with that split:

In [5]:
from datasets import load_dataset

data_files = {"train": "data/drugsComTrain_raw.tsv", "test": "data/drugsComTest_raw.tsv"}
drug_dataset = load_dataset("csv", data_files=data_files, delimiter="\t")

Using custom data configuration default-781d37ea3f205c6f
Found cached dataset csv (/home/chaklams/.cache/huggingface/datasets/csv/default-781d37ea3f205c6f/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/2 [00:00<?, ?it/s]

### Streaming datasets

To enable dataset streaming you just need to pass the `streaming=True` argument to the load_dataset() function. Streaming mode will make sure your PC won't have to load the whole dataset at once...

In [6]:
drug_dataset_streamed = load_dataset("csv", data_files=data_files, delimiter="\t", streaming=True)

Using custom data configuration default-781d37ea3f205c6f


Instead of the familiar `Dataset`, the object returned with `streaming=True` is an `IterableDataset`. As the name suggests, to access the elements of an `IterableDataset` we need to iterate over it. We can access the first element of our streamed dataset as follows:

In [7]:
type(drug_dataset_streamed)

datasets.dataset_dict.IterableDatasetDict

In [8]:
drug_dataset_streamed

{'train': <datasets.iterable_dataset.IterableDataset at 0x7f676cb10a90>,
 'test': <datasets.iterable_dataset.IterableDataset at 0x7f6766e695b0>}

In [9]:
next(iter(drug_dataset_streamed['train']))

/home/chaklams/.local/lib/python3.8/site-packages/datasets/download/streaming_download_manager.py:727: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)


{'Unnamed: 0': 206461,
 'drugName': 'Valsartan',
 'condition': 'Left Ventricular Dysfunction',
 'review': '"It has no side effect, I take it in combination of Bystolic 5 Mg and Fish Oil"',
 'rating': 9.0,
 'date': 'May 20, 2012',
 'usefulCount': 27}

The elements from a streamed dataset can be processed on the fly using `IterableDataset.map()`, which is useful during training if you need to tokenize the inputs.

In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
tokenized_dataset = drug_dataset_streamed.map(lambda x: tokenizer(x["review"]))

In [11]:
next(iter(tokenized_dataset['train']))

/home/chaklams/.local/lib/python3.8/site-packages/datasets/download/streaming_download_manager.py:727: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)


{'Unnamed: 0': 206461,
 'drugName': 'Valsartan',
 'condition': 'Left Ventricular Dysfunction',
 'review': '"It has no side effect, I take it in combination of Bystolic 5 Mg and Fish Oil"',
 'rating': 9.0,
 'date': 'May 20, 2012',
 'usefulCount': 27,
 'input_ids': [101,
  1000,
  2009,
  2038,
  2053,
  2217,
  3466,
  1010,
  1045,
  2202,
  2009,
  1999,
  5257,
  1997,
  2011,
  16033,
  10415,
  1019,
  11460,
  1998,
  3869,
  3514,
  1000,
  102],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

You can also shuffle a streamed dataset using `IterableDataset.shuffle()`, but unlike `Dataset.shuffle()` this only shuffles the elements in a predefined buffer_size:

In [12]:
shuffled_dataset = drug_dataset_streamed.shuffle(buffer_size=10_000, seed=42)
next(iter(shuffled_dataset['train']))

/home/chaklams/.local/lib/python3.8/site-packages/datasets/download/streaming_download_manager.py:727: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)


{'Unnamed: 0': 175494,
 'drugName': 'Scopolamine',
 'condition': 'Motion Sickness',
 'review': '"The reason I&#039;m giving it a 9 instead of 10 is because of it&#039;s side effects. Otherwise, this patch works really well for an overnight or two day fishing trip (at least in my experience). I thought I&#039;m getting cold and feeling sick after I get off the fishing boat because my throat was getting dry and dryer and dryer. At that time I&#039;m not aware that one of it&#039;s side effects was dry mouth/throat. Aside from that, I didn&#039;t feel anything anymore after couple of days. I&#039;ll use it again for my next fishing trip. I&#039;ll opt for a dry throat verses getting sick on the boat."',
 'rating': 9.0,
 'date': 'August 24, 2010',
 'usefulCount': 9}

In this example, we selected a random example from the first 10,000 examples in the buffer. Once an example is accessed, its spot in the buffer is filled with the next example in the corpus (i.e., the 10,001st example in the case above). You can also select elements from a streamed dataset using the `IterableDataset.take()` and `IterableDataset.skip()` functions, which act in a similar way to `Dataset.select()`. For example, to select the first 5 examples in the dataset we can do the following:

In [13]:
dataset_head = drug_dataset_streamed['train'].take(5)
list(dataset_head)

/home/chaklams/.local/lib/python3.8/site-packages/datasets/download/streaming_download_manager.py:727: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)


[{'Unnamed: 0': 206461,
  'drugName': 'Valsartan',
  'condition': 'Left Ventricular Dysfunction',
  'review': '"It has no side effect, I take it in combination of Bystolic 5 Mg and Fish Oil"',
  'rating': 9.0,
  'date': 'May 20, 2012',
  'usefulCount': 27},
 {'Unnamed: 0': 95260,
  'drugName': 'Guanfacine',
  'condition': 'ADHD',
  'review': '"My son is halfway through his fourth week of Intuniv. We became concerned when he began this last week, when he started taking the highest dose he will be on. For two days, he could hardly get out of bed, was very cranky, and slept for nearly 8 hours on a drive home from school vacation (very unusual for him.) I called his doctor on Monday morning and she said to stick it out a few days. See how he did at school, and with getting up in the morning. The last two days have been problem free. He is MUCH more agreeable than ever. He is less emotional (a good thing), less cranky. He is remembering all the things he should. Overall his behavior is bett

Similarly, you can use the `IterableDataset.skip()` function to create training and validation splits from a shuffled dataset as follows:

In [14]:
# Skip the first 1,000 examples and include the rest in the training set
train_dataset = shuffled_dataset['train'].skip(1000)
# Take the first 1,000 examples for the validation set
validation_dataset = shuffled_dataset['train'].take(1000)

For preprocessing part, we shall use the non-streamed version, since our data is not that big anyway...but remember to use streaming when your data is like >5GB..

## 2. Preprocessing

From this sample we can already see a few quirks in our dataset:

- The `Unnamed: 0` column looks suspiciously like an anonymized ID for each patient.
- The `condition` column includes a mix of uppercase and lowercase labels.
- The `reviews` are of varying length and contain a mix of Python line separators (\r\n) as well as HTML character codes like &\#039;.

### Rename patient id

Let’s see how we can use Datasets to deal with each of these issues. To test the patient ID hypothesis for the `Unnamed: 0` column, we can use the `Dataset.unique()` function to verify that the number of IDs matches the number of rows in each split:

In [15]:
for split in drug_dataset.keys():
    assert len(drug_dataset[split]) == len(drug_dataset[split].unique("Unnamed: 0"))

This seems to confirm our hypothesis, so let’s clean up the dataset a bit by renaming the Unnamed: 0 column to something a bit more interpretable. We can use the `DatasetDict.rename_column()` function to rename the column across both splits in one go:

In [16]:
drug_dataset = drug_dataset.rename_column(
    original_column_name="Unnamed: 0", new_column_name="patient_id"
)
drug_dataset

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 161297
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 53766
    })
})

### Lowercasing condition

Before we perform lowercasing, we got some problem with our `condition` since some is `None` (which cannot be lowercased). Let’s first drop these rows using `Dataset.filter()`, which works in a similar way to `Dataset.map()` and expects a function that receives a single example of the dataset.

In [17]:
drug_dataset = drug_dataset.filter(lambda x: x["condition"] is not None)

Loading cached processed dataset at /home/chaklams/.cache/huggingface/datasets/csv/default-781d37ea3f205c6f/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-e1e956c9dd698081.arrow
Loading cached processed dataset at /home/chaklams/.cache/huggingface/datasets/csv/default-781d37ea3f205c6f/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-7f566e92a92fcfa3.arrow


With the None entries removed, we can now normalize our `condition` column:

In [18]:
def lowercase_condition(example):
    return {"condition": example["condition"].lower()}

drug_dataset = drug_dataset.map(lowercase_condition)
# Check that lowercasing worked
drug_dataset["train"]["condition"][:3]

Loading cached processed dataset at /home/chaklams/.cache/huggingface/datasets/csv/default-781d37ea3f205c6f/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-aec27311b11c5a64.arrow
Loading cached processed dataset at /home/chaklams/.cache/huggingface/datasets/csv/default-781d37ea3f205c6f/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-55f2db5ea45774d7.arrow


['left ventricular dysfunction', 'adhd', 'birth control']

It works! Now that we’ve cleaned up the labels, let’s take a look at cleaning up the reviews themselves.

### Review length

Whenever you’re dealing with customer reviews, a good practice is to check the number of words in each review. A review might be just a single word like “Great!” or a full-blown essay with thousands of words, and depending on the use case you’ll need to handle these extremes differently. To compute the number of words in each review, we’ll use a rough heuristic based on splitting each text by whitespace.

Let’s define a simple function that counts the number of words in each review:

In [19]:
def compute_review_length(example):
    return {"review_length": len(example["review"].split())}

Unlike our `lowercase_condition()` function, `compute_review_length()` returns a dictionary whose key does not correspond to one of the column names in the dataset. In this case, when `compute_review_length()` is passed to `Dataset.map()`, it will be applied to all the rows in the dataset to create a new `review_length` column:

In [20]:
drug_dataset = drug_dataset.map(compute_review_length)
# Inspect the first training example
drug_dataset["train"][0]

Loading cached processed dataset at /home/chaklams/.cache/huggingface/datasets/csv/default-781d37ea3f205c6f/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-a38bf2ee84af9ffd.arrow
Loading cached processed dataset at /home/chaklams/.cache/huggingface/datasets/csv/default-781d37ea3f205c6f/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-32b756afafc251f7.arrow


{'patient_id': 206461,
 'drugName': 'Valsartan',
 'condition': 'left ventricular dysfunction',
 'review': '"It has no side effect, I take it in combination of Bystolic 5 Mg and Fish Oil"',
 'rating': 9.0,
 'date': 'May 20, 2012',
 'usefulCount': 27,
 'review_length': 17}

As expected, we can see a review_length column has been added to our training set. We can sort this new column with Dataset.sort() to see what the extreme values look like:

In [21]:
drug_dataset["train"].sort("review_length")[:3]

Loading cached sorted indices for dataset at /home/chaklams/.cache/huggingface/datasets/csv/default-781d37ea3f205c6f/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-5e9a51a8850f19c1.arrow


{'patient_id': [103488, 23627, 20558],
 'drugName': ['Loestrin 21 1 / 20', 'Chlorzoxazone', 'Nucynta'],
 'condition': ['birth control', 'muscle spasm', 'pain'],
 'review': ['"Excellent."', '"useless"', '"ok"'],
 'rating': [10.0, 1.0, 6.0],
 'date': ['November 4, 2008', 'March 24, 2017', 'August 20, 2016'],
 'usefulCount': [5, 2, 10],
 'review_length': [1, 1, 1]}

As we suspected, some reviews contain just a single word, which, although it may be okay for sentiment analysis, would not be informative if we want to predict the condition.

Note: An alternative way to add new columns to a dataset is with the `Dataset.add_column()` function. This allows you to provide the column as a Python list or NumPy array and can be handy in situations where Dataset.map() is not well suited for your analysis.

Let’s use the `Dataset.filter()` function to remove reviews that contain fewer than 30 words. Similarly to what we did with the `condition` column, we can filter out the very short reviews by requiring that the reviews have a length above this threshold:

In [22]:
drug_dataset = drug_dataset.filter(lambda x: x["review_length"] > 30)
print(drug_dataset.num_rows)

Loading cached processed dataset at /home/chaklams/.cache/huggingface/datasets/csv/default-781d37ea3f205c6f/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-ed2706c7bc365f9a.arrow
Loading cached processed dataset at /home/chaklams/.cache/huggingface/datasets/csv/default-781d37ea3f205c6f/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-aeee41f7dc4896e4.arrow


{'train': 138514, 'test': 46108}


As you can see, this has removed around 15% of the reviews from our original training and test sets.

### Escaping review html

The last thing we need to deal with is the presence of HTML character codes in our reviews. We can use Python’s `html` module to unescape these characters, like so:

In [23]:
import html

text = "I&#039;m a transformer called BERT"
html.unescape(text)

"I'm a transformer called BERT"

We’ll use `Dataset.map()` to unescape all the HTML characters in our corpus:

In [24]:
drug_dataset = drug_dataset.map(lambda x: {"review": html.unescape(x["review"])})

Loading cached processed dataset at /home/chaklams/.cache/huggingface/datasets/csv/default-781d37ea3f205c6f/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-00f0922078d99345.arrow
Loading cached processed dataset at /home/chaklams/.cache/huggingface/datasets/csv/default-781d37ea3f205c6f/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-e1342b3677faf817.arrow


As you can see, the `Dataset.map()` method is quite useful for processing data — and we haven’t even scratched the surface of everything it can do!

## 3. Faster preprocessing

### Batched=True

The `Dataset.map()` method takes a batched argument that, if set to `True`, causes it to send a batch of examples to the map function at once (the batch size is configurable but defaults to 1,000) which is faster.

When you specify `batched=True` the function receives a dictionary with the fields of the dataset, but each value is now a list of values, and not just a single value. The return value of `Dataset.map()` should be the same: a dictionary with the fields we want to update or add to our dataset, and a list of values. For example, here is another way to unescape all HTML characters, but using `batched=True`:

In [25]:
new_drug_dataset = drug_dataset.map(lambda x: {"review": html.unescape(x["review"])}, batched=True)

Loading cached processed dataset at /home/chaklams/.cache/huggingface/datasets/csv/default-781d37ea3f205c6f/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-be1beff3f18025aa.arrow
Loading cached processed dataset at /home/chaklams/.cache/huggingface/datasets/csv/default-781d37ea3f205c6f/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-6ef7168e831810d5.arrow


If you’re running this code in a notebook, you’ll see that this command executes way faster than the previous one.  This is 
because we gain some performance by accessing lots of elements at the same time instead of one by one.

### Fast tokenizers

Using `Dataset.map()` with `batched=True` will be essential to unlock the speed of the “fast” tokenizers that we’ll encounter later, which can quickly tokenize big lists of texts. Let's compare (1) slow vs. fast tokenizier, (2) batched vs no batched, and (3) num_proc vs no num_proc.

In [26]:
from transformers import AutoTokenizer

# #slow tokenizer + batched=True
# tokenizer = AutoTokenizer.from_pretrained("bert-base-cased", use_fast=False)

# def tokenize_function(examples):
#     return tokenizer(examples["review"], truncation=True)

# %time tokenized_dataset = drug_dataset.map(tokenize_function, batched=True)

In [27]:
# #slow tokenizer + batched=False
# tokenizer = AutoTokenizer.from_pretrained("bert-base-cased", use_fast=False)

# def tokenize_function(examples):
#     return tokenizer(examples["review"], truncation=True)

# %time tokenized_dataset = drug_dataset.map(tokenize_function, batched=False)

In [28]:
#fast tokenizer + batched=True
#recommended :)
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased", use_fast=True) #default is already True

def tokenize_function(examples):
    return tokenizer(examples["review"], truncation=True)

%time tokenized_dataset = drug_dataset.map(tokenize_function, batched=True)

Loading cached processed dataset at /home/chaklams/.cache/huggingface/datasets/csv/default-781d37ea3f205c6f/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-aa6d0b174526a4c1.arrow
Loading cached processed dataset at /home/chaklams/.cache/huggingface/datasets/csv/default-781d37ea3f205c6f/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-ec4be806df6da2a1.arrow


CPU times: user 11.9 ms, sys: 3.96 ms, total: 15.9 ms
Wall time: 15.5 ms


In [29]:
# #fast tokenizer + batched=False
# tokenizer = AutoTokenizer.from_pretrained("bert-base-cased", use_fast=True) #default is already True

# def tokenize_function(examples):
#     return tokenizer(examples["review"], truncation=True)

# %time tokenized_dataset = drug_dataset.map(tokenize_function, batched=False)

In [30]:
# #fast tokenizer + batched=False + num_proc=8
# tokenizer = AutoTokenizer.from_pretrained("bert-base-cased", use_fast=True) #default is already True

# def tokenize_function(examples):
#     return tokenizer(examples["review"], truncation=True)

# %time tokenized_dataset = drug_dataset.map(tokenize_function, batched=True, num_proc=8)

A fast tokenizer with the `batched=True` option is 30 times faster than its slow counterpart with no batching — this is truly amazing! Fast tokenizer is able to achieve such a speedup because behind the scenes the tokenization code is executed in Rust, which is a language that makes it easy to parallelize code execution.  Python is the language most often used for data science and deep learning applications, but when anything needs to be parallelized to be fast, it has to be written in another language. For instance, the matrix multiplications that are at the core of the model computation are written in CUDA, an optimized C library for GPUs.

Parallelization is also the reason for the nearly 6x speedup the fast tokenizer achieves with batching: you can’t parallelize a single tokenization operation, but when you want to tokenize lots of texts at the same time you can just split the execution across several processes, each responsible for its own texts.

`Dataset.map()` also has some parallelization capabilities of its own. Since they are not backed by Rust, they won’t let a slow tokenizer catch up with a fast one, but they can still be helpful (especially if you’re using a tokenizer that doesn’t have a fast version). To enable multiprocessing, use the `num_proc` argument and specify the number of processes to use in your call to Dataset.map()

### Maximum token length 

Most of the time, pretrained models support certain Here we will tokenize our examples and truncate them to a maximum length of 128, but we will ask the tokenizer to return all the chunks of the texts instead of just the first one. This can be done with `return_overflowing_tokens=True`:

In [31]:
def tokenize_and_split(examples):
    return tokenizer(
        examples["review"],
        truncation=True,
        max_length=128,
        return_overflowing_tokens=True,
    )

In [32]:
drug_dataset["train"][0]

{'patient_id': 95260,
 'drugName': 'Guanfacine',
 'condition': 'adhd',
 'review': '"My son is halfway through his fourth week of Intuniv. We became concerned when he began this last week, when he started taking the highest dose he will be on. For two days, he could hardly get out of bed, was very cranky, and slept for nearly 8 hours on a drive home from school vacation (very unusual for him.) I called his doctor on Monday morning and she said to stick it out a few days. See how he did at school, and with getting up in the morning. The last two days have been problem free. He is MUCH more agreeable than ever. He is less emotional (a good thing), less cranky. He is remembering all the things he should. Overall his behavior is better. \r\nWe have tried many different medications and so far this is the most effective."',
 'rating': 8.0,
 'date': 'April 27, 2010',
 'usefulCount': 192,
 'review_length': 141}

Let’s test this on one example before using `Dataset.map()` on the whole dataset:

In [33]:
result = tokenize_and_split(drug_dataset["train"][0])
result

{'input_ids': [[101, 107, 1422, 1488, 1110, 9079, 1194, 1117, 2223, 1989, 1104, 1130, 19972, 11083, 119, 1284, 1245, 4264, 1165, 1119, 1310, 1142, 1314, 1989, 117, 1165, 1119, 1408, 1781, 1103, 2439, 13753, 1119, 1209, 1129, 1113, 119, 1370, 1160, 1552, 117, 1119, 1180, 6374, 1243, 1149, 1104, 1908, 117, 1108, 1304, 172, 14687, 1183, 117, 1105, 7362, 1111, 2212, 129, 2005, 1113, 170, 2797, 1313, 1121, 1278, 12020, 113, 1304, 5283, 1111, 1140, 119, 114, 146, 1270, 1117, 3995, 1113, 6356, 2106, 1105, 1131, 1163, 1106, 6166, 1122, 1149, 170, 1374, 1552, 119, 3969, 1293, 1119, 1225, 1120, 1278, 117, 1105, 1114, 2033, 1146, 1107, 1103, 2106, 119, 1109, 1314, 1160, 1552, 1138, 1151, 2463, 1714, 119, 1124, 1110, 150, 21986, 3048, 1167, 5340, 1895, 1190, 1518, 102], [101, 119, 1124, 1110, 1750, 6438, 113, 170, 1363, 1645, 114, 117, 1750, 172, 14687, 1183, 119, 1124, 1110, 11566, 1155, 1103, 1614, 1119, 1431, 119, 8007, 1117, 4658, 1110, 1618, 119, 1284, 1138, 1793, 1242, 1472, 23897, 1105, 117

Notice that we got two `input_ids` list.  Let's try to decode what it is...

In [34]:
tokenizer.decode(result['input_ids'][0])

'[CLS] " My son is halfway through his fourth week of Intuniv. We became concerned when he began this last week, when he started taking the highest dose he will be on. For two days, he could hardly get out of bed, was very cranky, and slept for nearly 8 hours on a drive home from school vacation ( very unusual for him. ) I called his doctor on Monday morning and she said to stick it out a few days. See how he did at school, and with getting up in the morning. The last two days have been problem free. He is MUCH more agreeable than ever [SEP]'

In [35]:
tokenizer.decode(result['input_ids'][1])

'[CLS]. He is less emotional ( a good thing ), less cranky. He is remembering all the things he should. Overall his behavior is better. We have tried many different medications and so far this is the most effective. " [SEP]'

Now let’s do this for all elements of the dataset!

In [36]:
#this will give error
# tokenized_dataset = drug_dataset.map(tokenize_and_split, batched=True)

Oh no! That didn’t work! Why not? Looking at the error message will give us a clue: there is a mismatch in the lengths of one of the columns, one being of length 1,463 and the other of length 1,000. If you’ve looked at the `Dataset.map()` documentation, 1000 is the default batch size of `Dataset.map()`, while 1463 comes from chunking.

The problem is that we’re trying to mix two different datasets of different sizes: the drug_dataset columns will have a certain number of examples (the 1,000 in our error), but the tokenized_dataset we are building will have more (the 1,463 in the error message). That doesn’t work for a `Dataset`, so we need to either remove the columns from the old dataset or make them the same size as they are in the new dataset. We can do the former with the `remove_columns` argument:

In [37]:
drug_dataset["train"].column_names

['patient_id',
 'drugName',
 'condition',
 'review',
 'rating',
 'date',
 'usefulCount',
 'review_length']

In [38]:
tokenized_dataset = drug_dataset.map(
    tokenize_and_split, batched=True, remove_columns=drug_dataset["train"].column_names
)

Loading cached processed dataset at /home/chaklams/.cache/huggingface/datasets/csv/default-781d37ea3f205c6f/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-46a3cdc1f946b127.arrow
Loading cached processed dataset at /home/chaklams/.cache/huggingface/datasets/csv/default-781d37ea3f205c6f/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-be6754264493cf45.arrow


Now this works without error. We can check that our new dataset has many more elements than the original dataset by comparing the lengths:

In [39]:
#so now our tokenized dataset has no old columns, but only the tokenized info...
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'overflow_to_sample_mapping'],
        num_rows: 206772
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'overflow_to_sample_mapping'],
        num_rows: 68876
    })
})

In [40]:
#check that length got changed....
len(tokenized_dataset["train"]), len(drug_dataset["train"])

(206772, 138514)

We mentioned that we can also deal with the mismatched length problem by making the old columns the same size as the new ones. To do this, we will need the `overflow_to_sample_mapping` field the tokenizer returns when we set `return_overflowing_tokens=True`. It gives us a mapping from a new feature index to the index of the sample it originated from. Using this, we can associate each key present in our original dataset with a list of values of the right size by repeating the values of each example as many times as it generates new features:

In [41]:
def tokenize_and_split(examples):
    result = tokenizer(
        examples["review"],
        truncation=True,
        max_length=128,
        return_overflowing_tokens=True,
    )
    # Extract mapping between new and old indices
    sample_map = result.pop("overflow_to_sample_mapping")
    for key, values in examples.items():
        result[key] = [values[i] for i in sample_map]
    return result

We can see it works with Dataset.map() without us needing to remove the old columns:

In [42]:
tokenized_dataset = drug_dataset.map(tokenize_and_split, batched=True)
tokenized_dataset

Loading cached processed dataset at /home/chaklams/.cache/huggingface/datasets/csv/default-781d37ea3f205c6f/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-9bb0cca85ec5a0dd.arrow
Loading cached processed dataset at /home/chaklams/.cache/huggingface/datasets/csv/default-781d37ea3f205c6f/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-553abfd9825bbc81.arrow


DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 206772
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 68876
    })
})

We get the same number of training features as before, but here we’ve kept all the old fields. If you need them for some post-processing after applying your model, you might want to use this approach.

## 4. Using Pandas

You’ve now seen how 🤗 Datasets can be used to preprocess a dataset in various ways. Although the processing functions of 🤗 Datasets will cover most of your model training needs, there may be times when you’ll need to switch to Pandas to access more powerful features, like `DataFrame.groupby()` or high-level APIs for visualization. Fortunately, 🤗 Datasets is designed to be interoperable with libraries such as Pandas, NumPy, PyTorch, TensorFlow, and JAX. Let’s take a look at how this works.

To enable the conversion between various third-party libraries, 🤗 Datasets provides a `Dataset.set_format()` function. 

Under the hood, `Dataset.set_format()` changes the return format for the dataset’s __getitem__() dunder method. This means that when we want to create a new object like `train_df` from a Dataset in the "pandas" format, we need to slice the whole dataset to obtain a `pandas.DataFrame`. You can verify for yourself that the type of drug_dataset["train"] is Dataset, irrespective of the output format.

This function only changes the output format of the dataset, so you can easily switch to another format without affecting the underlying data format, which is Apache Arrow. The formatting is done in place. To demonstrate, let’s convert our dataset to Pandas:

In [43]:
drug_dataset.set_format("pandas") #inplace

In [44]:
#check that drug_dataset is still Dataset
type(drug_dataset)

datasets.dataset_dict.DatasetDict

Now when we access elements of the dataset we get a `pandas.DataFrame` instead of a dictionary:

In [45]:
print(type(drug_dataset["train"][:3]))
drug_dataset["train"][:10]

<class 'pandas.core.frame.DataFrame'>


,patient_id,drugName,condition,review,rating,date,usefulCount,review_length
0,95260,Guanfacine,adhd,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192,141
1,92703,Lybrel,birth control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17,134
2,138000,Ortho Evra,birth control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10,89
3,35696,Buprenorphine / naloxone,opiate dependence,"""Suboxone has completely turned my life around...",9.0,"November 27, 2016",37,124
4,155963,Cialis,benign prostatic hyperplasia,"""2nd day on 5mg started to work with rock hard...",2.0,"November 28, 2015",43,68
5,102654,Aripiprazole,bipolar disorde,"""Abilify changed my life. There is hope. I was...",10.0,"March 14, 2015",32,146
6,74811,Keppra,epilepsy,""" I Ve had nothing but problems with the Kepp...",1.0,"August 9, 2016",11,35
7,48928,Ethinyl estradiol / levonorgestrel,birth control,"""I had been on the pill for many years. When m...",8.0,"December 8, 2016",1,142
8,29607,Topiramate,migraine prevention,"""I have been on this medication almost two wee...",9.0,"January 1, 2015",19,148
9,75612,L-methylfolate,depression,"""I have taken anti-depressants for years, with...",10.0,"March 9, 2017",54,80


Let’s create a pandas.DataFrame for the whole training set by selecting all the elements of `drug_dataset["train"]`:

In [46]:
train_df = drug_dataset["train"][:]

From here we can use all the Pandas functionality that we want. 

In [47]:
train_df.head()

,patient_id,drugName,condition,review,rating,date,usefulCount,review_length
0,95260,Guanfacine,adhd,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192,141
1,92703,Lybrel,birth control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17,134
2,138000,Ortho Evra,birth control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10,89
3,35696,Buprenorphine / naloxone,opiate dependence,"""Suboxone has completely turned my life around...",9.0,"November 27, 2016",37,124
4,155963,Cialis,benign prostatic hyperplasia,"""2nd day on 5mg started to work with rock hard...",2.0,"November 28, 2015",43,68


And once we’re done with our Pandas analysis, we can always create a new Dataset object by using the `Dataset.from_pandas()` function as follows:

In [48]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_df)
train_dataset

Dataset({
    features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
    num_rows: 138514
})

## 5. Creating a validation set

To round out the section, let’s create a validation set to prepare the dataset for training a classifier on. Before doing so, we’ll reset the output format of drug_dataset from "pandas" to "arrow":

In [49]:
drug_dataset.reset_format()

Although we have a test set we could use for evaluation, it’s a good practice to leave the test set untouched and create a separate validation set during development. Once you are happy with the performance of your models on the validation set, you can do a final sanity check on the test set. This process helps mitigate the risk that you’ll overfit to the test set and deploy a model that fails on real-world data.

🤗 Datasets provides a `Dataset.train_test_split()` function that is based on the famous functionality from scikit-learn. Let’s use it to split our training set into train and validation splits (we set the seed argument for reproducibility):

In [50]:
drug_dataset

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 138514
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 46108
    })
})

In [51]:
drug_dataset_clean = drug_dataset["train"].train_test_split(train_size=0.8, seed=42)
drug_dataset_clean

Loading cached split indices for dataset at /home/chaklams/.cache/huggingface/datasets/csv/default-781d37ea3f205c6f/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-a2aff6acd88adfb9.arrow and /home/chaklams/.cache/huggingface/datasets/csv/default-781d37ea3f205c6f/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-9bf84c86a3e05a56.arrow


DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 110811
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 27703
    })
})

In [52]:
# Rename the default "test" split to "validation"
drug_dataset_clean["validation"] = drug_dataset_clean.pop("test")
drug_dataset_clean

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 110811
    })
    validation: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 27703
    })
})

In [53]:
# Add the "test" set to our `DatasetDict`
drug_dataset_clean["test"] = drug_dataset["test"]
drug_dataset_clean

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 110811
    })
    validation: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 27703
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 46108
    })
})

## Saving a dataset

Finally, after preprocessing, we may want to save the `Dataset`.  Huggingface provides `Dataset.save_to_disk()` (Apache Arrow), `Dataset.to_csv` (csv) and `Dataset.to_json()` (JSON)

For example, let’s save our cleaned dataset in the Arrow format:

In [54]:
drug_dataset_clean.save_to_disk("drug-reviews")

Loading cached processed dataset at /home/chaklams/.cache/huggingface/datasets/csv/default-781d37ea3f205c6f/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-b8edfaec310f7fab.arrow


Saving the dataset (0/1 shards):   0%|          | 0/110811 [00:00<?, ? examples/s]

Loading cached processed dataset at /home/chaklams/.cache/huggingface/datasets/csv/default-781d37ea3f205c6f/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-4792be80ca98a56e.arrow


Saving the dataset (0/1 shards):   0%|          | 0/27703 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/46108 [00:00<?, ? examples/s]

Notice you got some new dir.  We can load it by using the `load_from_disk()` function as follows:

In [55]:
from datasets import load_from_disk

drug_dataset_reloaded = load_from_disk("drug-reviews")
drug_dataset_reloaded

/home/chaklams/.local/lib/python3.8/site-packages/datasets/dataset_dict.py:1241: FutureWarning: 'fs' was is deprecated in favor of 'storage_options' in version 2.8.0 and will be removed in 3.0.0.
You can remove this warning by passing 'storage_options=fs.storage_options' instead.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 110811
    })
    validation: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 27703
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 46108
    })
})

For the CSV and JSON formats, we have to store each split as a separate file. One way to do this is by iterating over the keys and values in the DatasetDict object:

In [56]:
for split, dataset in drug_dataset_clean.items():
    dataset.to_json(f"drug-reviews-{split}.jsonl")

Creating json from Arrow format:   0%|          | 0/111 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/28 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/47 [00:00<?, ?ba/s]

This saves each split in JSON Lines format, where each row in the dataset is stored as a single line of JSON. Here’s what the first example looks like:

In [57]:
!head -n 1 drug-reviews-train.jsonl

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
{"patient_id":89879,"drugName":"Cyclosporine","condition":"keratoconjunctivitis sicca","review":"\"I have used Restasis for about a year now and have seen almost no progress.  For most of my life I've had red and bothersome eyes. After trying various eye drops, my doctor recommended Restasis.  He said it typically takes 3 to 6 months for it to really kick in but it never did kick in.  When I put the drops in it burns my eyes for the first 30 - 40 minutes.  I've talked with my doctor about this and he said it is normal but should go away after some time, but it hasn't. Every year around spring time my eyes get terrible irritated  and this year has been the same (maybe 

We can load the json file using `load_dataset`.

In [58]:
data_files = {
    "train": "drug-reviews-train.jsonl",
    "validation": "drug-reviews-validation.jsonl",
    "test": "drug-reviews-test.jsonl",
}
drug_dataset_reloaded = load_dataset("json", data_files=data_files)

Using custom data configuration default-bdae8a4a887d0ba9


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /home/chaklams/.cache/huggingface/datasets/json/default-bdae8a4a887d0ba9/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]